In [1]:
#Import geopandas to work with vector formats
import geopandas as gpd

boundarypath = "./kathmandu.geojson"
kathmandu = gpd.read_file(boundarypath)

#Change projection to WGS84 and get the coordinates for the bounding box
# campinas.crs = 'EPSG:4326'
envelope = kathmandu.geometry.total_bounds

In [2]:
kathmandu

,zona,SubClasses,ExtendedEn,Linetype,EntityHand,Text,geometry
0,ZAE B,AcDbEntity:AcDbPolyline,NaN,NaN,208,NaN,"POLYGON ((-47.09085 -23.04629, -47.08475 -23.0..."
1,0,AcDbEntity:AcDbPolyline,NaN,NaN,209,NaN,"POLYGON ((-47.11024 -23.04330, -47.11157 -23.0..."
2,ZAE A,AcDbEntity:AcDbPolyline,NaN,NaN,20A,NaN,"POLYGON ((-47.11157 -23.04426, -47.11024 -23.0..."
3,ZM 1,AcDbEntity:AcDbPolyline,NaN,NaN,20B,NaN,"POLYGON ((-47.10949 -23.04376, -47.11024 -23.0..."
4,ZAE A,AcDbEntity:AcDbPolyline,NaN,NaN,20C,NaN,"POLYGON ((-47.13450 -23.04288, -47.13455 -23.0..."
...,...,...,...,...,...,...,...
182,ZM 2,AcDbEntity:AcDbPolyline,NaN,NaN,38A,NaN,"POLYGON ((-47.08255 -22.96466, -47.08376 -22.9..."
183,ZM 2,AcDbEntity:AcDbPolyline,NaN,NaN,399,NaN,"POLYGON ((-47.07089 -22.94088, -47.06986 -22.9..."
184,ZAE B,AcDbEntity:AcDbPolyline,NaN,NaN,3A9,NaN,"POLYGON ((-47.13729 -22.89551, -47.13701 -22.8..."
185,ZM 2,AcDbEntity:AcDbPolyline,NaN,NaN,3AC,NaN,"POLYGON ((-47.10945 -22.90235, -47.10677 -22.9..."


In [3]:
kathmandu['geometry']

0      POLYGON ((-47.09085 -23.04629, -47.08475 -23.0...
1      POLYGON ((-47.11024 -23.04330, -47.11157 -23.0...
2      POLYGON ((-47.11157 -23.04426, -47.11024 -23.0...
3      POLYGON ((-47.10949 -23.04376, -47.11024 -23.0...
4      POLYGON ((-47.13450 -23.04288, -47.13455 -23.0...
                             ...                        
182    POLYGON ((-47.08255 -22.96466, -47.08376 -22.9...
183    POLYGON ((-47.07089 -22.94088, -47.06986 -22.9...
184    POLYGON ((-47.13729 -22.89551, -47.13701 -22.8...
185    POLYGON ((-47.10945 -22.90235, -47.10677 -22.9...
186    POLYGON ((-47.12445 -22.96248, -47.12490 -22.9...
Name: geometry, Length: 187, dtype: geometry

In [4]:
envelope

array([-47.22665286, -23.05538417, -46.99197819, -22.72705953])

In [6]:
#Import Google Earth Engine and initialize it
import ee
ee.Initialize()
ee.Authenticate()
aoi=ee.Geometry.Rectangle([-47.22665286, -23.05538417, -46.99197819, -22.72705953])

#Access the Landsat8 collection and apply a filter for cloud cover at no more than 10%
collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').filterBounds(aoi)
collection = collection.filterDate('2015-11-01', '2022-04-30').filterMetadata('CLOUD_COVER', 'less_than', 10)

#Filter the collection for just the RGB and Thermal bands
collection = collection.select('SR_B4', 'SR_B3', 'SR_B2', 'ST_B10')


ConnectionError: HTTPSConnectionPool(host='earthengine.googleapis.com', port=443): Max retries exceeded with url: /$discovery/rest?version=v1alpha&prettyPrint=false (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f7e13994e80>: Failed to establish a new connection: [Errno 101] Network is unreachable'))

In [62]:
#Calculate the median value for each pixel in the collection of images and clip it to the area of interest
median = collection.reduce(ee.Reducer.median()).clip(aoi)

#Apply a scale factor of 0.00341802, an additional offset of 149.0 and a subtraction of 273.15 per pixel to get values in degrees Celsius
#LST = B10 ∗ 0.00341802 + 149 − 273.15

#Apply the expression and create a new band ST_B10_median for the results
LST_median = median.expression('(B10*0.00341802) -124.15', {
      'B10': median.select('ST_B10_median')
});

In [63]:
sim_geo = gpd.GeoSeries(kathmandu['geometry']).simplify(tolerance=0.001)
geo_j = sim_geo.to_json()

In [64]:
#Import the libraries
import folium
import geehydro

#Create a Folium map object
Mapa = folium.Map(location=[27.700769, 85.300140], zoom_start=9)

#Declare the parameters for the raster
lstpalette = ['red', 'green', 'blue']

lst_parameters = {'min':15,
                  'max':45,
                  'dimensions':512,
                  'palette': lstpalette,
                  'region':aoi  
}

#Add our LST raster to the Folium map
Mapa.addLayer(LST_median, lst_parameters)
folium.GeoJson(geo_j, name="geojson").add_to(Mapa)
#Visualize it on the cell
Mapa

In [34]:
ee.batch.Export.image.toDrive(
            image = LST_median.clip(aoi),
            folder = 'GEE export',
            fileNamePrefix = 'Landsat8_',
            scale = 30).start()